In [1]:
#import required libraries 

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [0]:
import os

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
#go to working directory
cd "/content/drive/My Drive/ColabNotebooks/PicToRecipe"

/content/drive/My Drive/ColabNotebooks/PicToRecipe


In [0]:
#define the veriables
img_width, img_height = 224, 224
train_data_dir = "DataSet/Training_set"
validation_data_dir = "DataSet/Test_set"

In [6]:
path, data_dir_list, files = next(os.walk(train_data_dir))
data_dir_list

['Red Tomato',
 'Broccoli',
 'Spring Onion',
 'Mushroom',
 'Potato',
 'Carrot',
 'Corn',
 'Spinach',
 'Eggplant',
 'Onion',
 'Coliflower',
 'Green Tomato',
 'Brussels Sprouts',
 'Green Beans',
 'Avocado']

In [7]:
#Calculate Train samples
train_samples = 0

path, dirs, files = next(os.walk(train_data_dir))
nb_classsifires = len(dirs)
print(nb_classsifires)
for d in dirs:
    path, nub_classifires, files = next(os.walk(train_data_dir + "/" + d))
    train_samples = train_samples+len(files)
    
print(train_samples ) 

15
623


In [8]:
test_samples = 0

path, dirs, files = next(os.walk(validation_data_dir))
for d in dirs:
    path, nub_classifires, files = next(os.walk(validation_data_dir + "/" + d))
    test_samples = test_samples+len(files)
    
print(test_samples) 

202


In [0]:
batch_size = 10
epochs = 25

In [10]:
#VGG19 model, with weights pre-trained on ImageNet.
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#model = applications.VGG19(weights = "imagenet", include_top=True, classes=1000, input_shape = None)

80142336/80134624 [==============================] - 6s 0us/step


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:4]:
    layer.trainable = False

In [0]:
#from keras.preprocessing.image import img_to_array

In [0]:
#x = model.get_layer('fc2').output
#x = Dense(128, activation="relu")(x)
#x = Dropout(0.5)(x)
#x = Dense(128, activation="relu")(x)
#x = Dropout(0.2)(x)
#x = Dense(128, activation="relu")(x)
#predictions = Dense(nb_classsifires, activation="softmax")(x)

In [0]:
x = model.output
x = Flatten()(x) 
x = Dense(128, activation="relu")(x) 
x = Dropout(0.5)(x) 
x = Dense(128, activation="relu")(x) 
x = Dropout(0.2)(x) 
x = Dense(128, activation="relu")(x) 
predictions = Dense(nb_classsifires, activation="softmax")(x)

In [0]:
# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

In [0]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [0]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

In [0]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30)

In [20]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, img_width),
                                                    batch_size = batch_size, 
                                                    class_mode = "categorical")

Found 621 images belonging to 15 classes.


In [21]:
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_height, img_width),
                                                        class_mode = "categorical")

Found 201 images belonging to 15 classes.


In [22]:
train_generator.class_indices

{'Avocado': 0,
 'Broccoli': 1,
 'Brussels Sprouts': 2,
 'Carrot': 3,
 'Coliflower': 4,
 'Corn': 5,
 'Eggplant': 6,
 'Green Beans': 7,
 'Green Tomato': 8,
 'Mushroom': 9,
 'Onion': 10,
 'Potato': 11,
 'Red Tomato': 12,
 'Spinach': 13,
 'Spring Onion': 14}

In [0]:
import pandas as pd
d = train_generator.class_indices

In [45]:
d.items()

dict_items([('Avocado', 0), ('Broccoli', 1), ('Brussels Sprouts', 2), ('Carrot', 3), ('Coliflower', 4), ('Corn', 5), ('Eggplant', 6), ('Green Beans', 7), ('Green Tomato', 8), ('Mushroom', 9), ('Onion', 10), ('Potato', 11), ('Red Tomato', 12), ('Spinach', 13), ('Spring Onion', 14)])

In [0]:
labels = pd.DataFrame(list(d.items()), columns=['Label', 'Index'])

In [53]:
labels

,Label,Index
0,Avocado,0
1,Broccoli,1
2,Brussels Sprouts,2
3,Carrot,3
4,Coliflower,4
5,Corn,5
6,Eggplant,6
7,Green Beans,7
8,Green Tomato,8
9,Mushroom,9


In [0]:
labels.to_csv("labels.csv", index=False)

In [0]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg19_pictoreceipe_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [0]:
# Train the model 
model_final.fit_generator(train_generator,
                          steps_per_epoch = train_samples/2,
                          epochs = epochs,
                          validation_data = validation_generator,
                          validation_steps = test_samples,
                          callbacks = [checkpoint, early])

Epoch 1/25
312/311 [==============================] - 540s 2s/step - loss: 2.6934 - acc: 0.1074 - val_loss: 2.6783 - val_acc: 0.0656

Epoch 00001: val_acc improved from -inf to 0.06564, saving model to vgg19_pictoreceipe_1.h5
Epoch 2/25
312/311 [==============================] - 236s 755ms/step - loss: 2.5717 - acc: 0.1760 - val_loss: 2.4254 - val_acc: 0.2125

Epoch 00002: val_acc improved from 0.06564 to 0.21252, saving model to vgg19_pictoreceipe_1.h5
Epoch 3/25
312/311 [==============================] - 236s 756ms/step - loss: 2.0879 - acc: 0.3295 - val_loss: 1.7305 - val_acc: 0.4490

Epoch 00003: val_acc improved from 0.21252 to 0.44903, saving model to vgg19_pictoreceipe_1.h5
Epoch 4/25
312/311 [==============================] - 236s 757ms/step - loss: 1.5822 - acc: 0.4824 - val_loss: 1.2518 - val_acc: 0.6358

Epoch 00004: val_acc improved from 0.44903 to 0.63585, saving model to vgg19_pictoreceipe_1.h5
Epoch 5/25
312/311 [==============================] - 236s 756ms/step - loss: 